In [10]:
# Import
import os

import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import skimage
import skimage.transform
import pandas as pd

In [5]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhyGJ1otRDKv-P-Ljue0SMWw847Mgf_Oey1HExAAqYGymKQG-T12cE

Successfully saved authorization token.


In [7]:
def get_ee_image_S2(long, lat, start_date = '2020-09-01', end_date = '2021-09-21', band_list = ['B4', 'B3', 'B2']):
    """
    long = float (incoming longitute of landfill)
    lat = float (incoming lattitude of landfill)
    start_date = string (start date for image sampling)
    end_date = string (end date for image sampling)
    band_list = list(string) (list of band strings to be extracted)
    
    return geemap image object given the long, lat
    """
    # Half width of geometry region
    _HW = 0.090909

    point = ee.Geometry.Point(long, lat)
    aoi = ee.Geometry.Polygon([[
        [long - _HW, lat + _HW],
        [long - _HW, lat - _HW],
        [long + _HW, lat - _HW],
        [long + _HW, lat + _HW],
    ]], None, False)

    img_col = (
        ee
        .ImageCollection('COPERNICUS/S2_SR') # load from the Sentinel2 data source
        .filterBounds(point)
        .filterDate(start_date, end_date) 
        .sort('CLOUDY_PIXEL_PERCENTAGE') # sort ascending by the cloudy pixel percentage
        .select(band_list)
    )
    
    num_imgs = img_col.size().getInfo()
    if num_imgs == 0:
        return None

    img = (
        img_col
        .first() # get the lest cloudy image
        .clipToBoundsAndScale(
                geometry=aoi,
                width=512,
                height=512,
            )
    )

    return img

def save_img(img, path, img_filename):
    
    filepath = os.path.join(path, img_filename)
    
    try: 
        img_tensor = geemap.ee_to_numpy(img)
        img_tensor = img_tensor.clip(0, 3000) / 3000
    
    except:
        print("Skipping: image not found")
        return
    
    plt.imsave(f'{filepath}.png', img_tensor)
    return 

In [4]:
# https://simplemaps.com/data/world-cities

countryToSample = 'India'

citiesDf = pd.read_csv('Data/worldcities.csv')
citiesDf['latlong'] = list(zip(citiesDf['lat'], citiesDf['lng']))
citiesLatLongs = pd.DataFrame(citiesDf[['city', 'latlong']])
countryCitiesDf = citiesDf.loc[citiesDf['country'] == countryToSample]
countryCitiesDf.head(20)

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,latlong
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604,"(28.66, 77.23)"
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629,"(18.9667, 72.8333)"
13,Kolkāta,Kolkata,22.5411,88.3378,India,IN,IND,West Bengal,admin,17560000.0,1356060520,"(22.5411, 88.3378)"
23,Bangalore,Bangalore,12.9699,77.5980,India,IN,IND,Karnātaka,admin,13707000.0,1356410365,"(12.9699, 77.598)"
31,Chennai,Chennai,13.0825,80.2750,India,IN,IND,Tamil Nādu,admin,11324000.0,1356374944,"(13.0825, 80.275)"
41,Hyderābād,Hyderabad,17.3667,78.4667,India,IN,IND,Telangana,admin,9746000.0,1356871768,"(17.3667, 78.4667)"
60,Pune,Pune,18.5196,73.8553,India,IN,IND,Mahārāshtra,NaN,7764000.0,1356081074,"(18.5196, 73.8553)"
68,Ahmedabad,Ahmedabad,23.0300,72.5800,India,IN,IND,Gujarāt,minor,7410000.0,1356304381,"(23.03, 72.58)"
106,Sūrat,Surat,21.1700,72.8300,India,IN,IND,Gujarāt,NaN,5807000.0,1356758738,"(21.17, 72.83)"
235,Lucknow,Lucknow,26.8470,80.9470,India,IN,IND,Uttar Pradesh,admin,3382000.0,1356891790,"(26.847, 80.947)"


In [7]:
import time

savePath = ''
halfBoxKm = 20
latSteps = np.arange(-halfBoxKm/110, halfBoxKm/110, .024) # 110 is hardcoded for quick conversion of 1deg lat to km (not exact)

tStart = time.time()

imgCounter = 1

for cityNum in np.arange(10): # np.arange(len(countryCitiesDf)):
    _lng = countryCitiesDf.iloc[cityNum]['lng']
    _lat = countryCitiesDf.iloc[cityNum]['lat']
    
    for latStepSize in latSteps:
        longDegInKm = 111*np.cos((_lat + latStepSize)*np.pi/180)
        longSteps = np.arange(-halfBoxKm/longDegInKm, halfBoxKm/longDegInKm, .024)
        
        for longStepSize in longSteps:
            saveFName = countryToSample.upper() + '_' + countryCitiesDf.iloc[cityNum]['city'] + '_' + str(np.round(_lat + latStepSize, 3)) + '_' + str(np.round(_lng + longStepSize, 3))

            _img = get_ee_image_S2(_lng + longStepSize, _lat + latStepSize)
            save_img(img=_img, path=savePath, img_filename=saveFName)

            imgCounter += 1
    
tEnd = time.time()
print(tEnd - tStart)

Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B3' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().
Skipping: image not found
Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B4' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().
Skipping: image not found
Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B2' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().
Skipping: image not found
Image.sampleRectangle: Fully masked pixels / pixels outside of the image footpr